In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def actor(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('actor', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def generator(actions, state_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=actions, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=state_size)        
        return logits

In [6]:
def discriminator(states, actions, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        return logits

In [10]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    actions_logits = actor(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ###############################################
    next_states_logits = generator(actions=actions_logits, hidden_size=hidden_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    aloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits, 
                                                                    labels=next_states_labels))
    ####################################################
    dQs = discriminator(actions=actions_labels, hidden_size=hidden_size, states=states, action_size=action_size)
    rates = tf.reshape(rates, shape=[-1, 1])
    dloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dQs, # GAN
                                                                   labels=rates)) # 0-1
    ####################################################
    gQs = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states, action_size=action_size, 
                        reuse=True)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.zeros_like(gQs))) # 0-1
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.ones_like(gQs))) # 0-1
    #####################################################
    next_actions_logits = actor(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    gQs2 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states, 
                         action_size=action_size, reuse=True)
    gQs2 = tf.reshape(gQs2, shape=[-1]) * (1-dones)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                    labels=tf.zeros_like(gQs2))) # 0-1
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                     labels=tf.ones_like(gQs2))) # 0-1
    return actions_logits, aloss, dloss, aloss2, (1-dones)

In [11]:
def model_opt(a_loss, a_loss2, d_loss, a_learning_rate, d_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('actor')]
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        d_opt = tf.train.AdamOptimizer(d_learning_rate).minimize(d_loss, var_list=d_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, d_opt, a_opt2

In [12]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, d_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.d_loss, self.a_loss2, self.dones__ = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, #model input 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.d_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        a_loss2=self.a_loss2, 
                                                        d_loss=self.d_loss,
                                                        a_learning_rate=a_learning_rate,
                                                        d_learning_rate=d_learning_rate)

In [13]:
# def sample(buffer, batch_size):
#     idx = np.random.choice(np.arange(len(buffer)), size=batch_size, replace=False)
#     return [buffer[ii] for ii in idx]

In [14]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [15]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [16]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
d_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size: 200/500 a successfull episode size
gamma = 0.99                   # future reward discount

In [17]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              a_learning_rate=a_learning_rate, d_learning_rate=d_learning_rate, gamma=gamma)

# Init the memory
memory = Memory(max_size=memory_size)

In [18]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        state = env.reset()
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
# aloss_list, dloss_list, aloss2_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0 # each episode
        aloss_batch, dloss_batch, aloss2_batch = [], [], []
        state = env.reset() # each episode
        num_step = 0 # each episode

        # Training steps/batches
        while True:
            # Explore (Env) or Exploit (Model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            rate = -1
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Rating the memory
            if done is True:
                rate = total_reward/500 # update rate at the end/ when episode is done
                for idx in range(num_step): # episode length
                    if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                        memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                        
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            idx = np.random.choice(np.arange(memory_size// batch_size))
            states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            states = states[rates == np.max(rates)]
            actions = actions[rates == np.max(rates)]
            next_states = next_states[rates == np.max(rates)]
            rewards = rewards[rates == np.max(rates)]
            dones = dones[rates == np.max(rates)]
            rates = rates[rates == np.max(rates)]
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            dloss, _ = sess.run([model.d_loss, model.d_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _ = sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            #print(dones__, np.max(rates), rates.shape, dones)
            aloss_batch.append(aloss)
            dloss_batch.append(dloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'dloss:{:.4f}'.format(np.mean(dloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        # gloss_list.append([ep, np.mean(gloss_batch)])
        # dloss_list.append([ep, np.mean(dloss_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:15.0000 R:15.0000 rate:0.0300 aloss:1.3947 dloss:1.9470 aloss2:1.4562 exploreP:0.9985
Episode:1 meanR:18.5000 R:22.0000 rate:0.0440 aloss:1.3930 dloss:1.9546 aloss2:1.4466 exploreP:0.9963
Episode:2 meanR:17.6667 R:16.0000 rate:0.0320 aloss:1.4193 dloss:1.8832 aloss2:1.5611 exploreP:0.9948
Episode:3 meanR:17.2500 R:16.0000 rate:0.0320 aloss:1.3846 dloss:1.9181 aloss2:1.4775 exploreP:0.9932
Episode:4 meanR:16.0000 R:11.0000 rate:0.0220 aloss:1.3852 dloss:1.9609 aloss2:1.4239 exploreP:0.9921
Episode:5 meanR:16.1667 R:17.0000 rate:0.0340 aloss:1.3910 dloss:1.9152 aloss2:1.4811 exploreP:0.9904
Episode:6 meanR:18.4286 R:32.0000 rate:0.0640 aloss:1.3963 dloss:1.8672 aloss2:1.5399 exploreP:0.9873
Episode:7 meanR:17.8750 R:14.0000 rate:0.0280 aloss:1.3834 dloss:1.8591 aloss2:1.5260 exploreP:0.9859
Episode:8 meanR:17.5556 R:15.0000 rate:0.0300 aloss:1.3974 dloss:1.8663 aloss2:1.5395 exploreP:0.9845
Episode:9 meanR:17.7000 R:19.0000 rate:0.0380 aloss:1.3925 dloss:1.8956 aloss2:1.4

Episode:80 meanR:21.7284 R:11.0000 rate:0.0220 aloss:1.3770 dloss:0.7764 aloss2:3.6627 exploreP:0.8402
Episode:81 meanR:21.6341 R:14.0000 rate:0.0280 aloss:1.3754 dloss:0.8457 aloss2:3.4774 exploreP:0.8391
Episode:82 meanR:21.5422 R:14.0000 rate:0.0280 aloss:1.3863 dloss:0.8359 aloss2:3.4722 exploreP:0.8379
Episode:83 meanR:21.4762 R:16.0000 rate:0.0320 aloss:1.3772 dloss:0.8092 aloss2:3.5920 exploreP:0.8366
Episode:84 meanR:21.4000 R:15.0000 rate:0.0300 aloss:1.3878 dloss:0.8488 aloss2:3.3494 exploreP:0.8353
Episode:85 meanR:21.3372 R:16.0000 rate:0.0320 aloss:1.3708 dloss:0.7995 aloss2:3.7545 exploreP:0.8340
Episode:86 meanR:21.3448 R:22.0000 rate:0.0440 aloss:1.3801 dloss:0.7875 aloss2:3.7692 exploreP:0.8322
Episode:87 meanR:21.5795 R:42.0000 rate:0.0840 aloss:1.3769 dloss:0.7857 aloss2:3.7975 exploreP:0.8288
Episode:88 meanR:21.5056 R:15.0000 rate:0.0300 aloss:1.3740 dloss:0.7753 aloss2:3.7515 exploreP:0.8275
Episode:89 meanR:21.4111 R:13.0000 rate:0.0260 aloss:1.3818 dloss:0.8011 

Episode:159 meanR:31.9000 R:27.0000 rate:0.0540 aloss:1.3735 dloss:0.6054 aloss2:6.7721 exploreP:0.6421
Episode:160 meanR:32.6400 R:99.0000 rate:0.1980 aloss:1.3710 dloss:0.5632 aloss2:6.1142 exploreP:0.6358
Episode:161 meanR:32.6000 R:13.0000 rate:0.0260 aloss:1.3702 dloss:0.5696 aloss2:5.8815 exploreP:0.6350
Episode:162 meanR:33.0400 R:55.0000 rate:0.1100 aloss:1.3657 dloss:0.5421 aloss2:6.0137 exploreP:0.6316
Episode:163 meanR:33.2000 R:28.0000 rate:0.0560 aloss:1.3708 dloss:0.5953 aloss2:6.6354 exploreP:0.6299
Episode:164 meanR:33.5400 R:53.0000 rate:0.1060 aloss:1.3636 dloss:0.5881 aloss2:6.3328 exploreP:0.6266
Episode:165 meanR:33.2900 R:51.0000 rate:0.1020 aloss:1.3654 dloss:0.5636 aloss2:6.2154 exploreP:0.6235
Episode:166 meanR:33.2400 R:12.0000 rate:0.0240 aloss:1.3620 dloss:0.5795 aloss2:6.3109 exploreP:0.6227
Episode:167 meanR:33.5200 R:45.0000 rate:0.0900 aloss:1.3616 dloss:0.5783 aloss2:6.2585 exploreP:0.6200
Episode:168 meanR:33.7000 R:37.0000 rate:0.0740 aloss:1.3624 dlo

Episode:238 meanR:126.4400 R:500.0000 rate:1.0000 aloss:1.3640 dloss:0.7400 aloss2:5.4229 exploreP:0.2067
Episode:239 meanR:130.7600 R:500.0000 rate:1.0000 aloss:1.3655 dloss:0.7689 aloss2:5.3586 exploreP:0.1971
Episode:240 meanR:133.3800 R:351.0000 rate:0.7020 aloss:1.3656 dloss:0.7544 aloss2:5.3406 exploreP:0.1907
Episode:241 meanR:135.3300 R:275.0000 rate:0.5500 aloss:1.3666 dloss:0.7006 aloss2:5.4215 exploreP:0.1858
Episode:242 meanR:139.5000 R:500.0000 rate:1.0000 aloss:1.3630 dloss:0.7830 aloss2:5.3755 exploreP:0.1772
Episode:243 meanR:141.5700 R:252.0000 rate:0.5040 aloss:1.3613 dloss:0.7409 aloss2:5.3548 exploreP:0.1730
Episode:244 meanR:146.4300 R:500.0000 rate:1.0000 aloss:1.3621 dloss:0.8028 aloss2:5.2907 exploreP:0.1651
Episode:245 meanR:149.2800 R:304.0000 rate:0.6080 aloss:1.3628 dloss:0.7811 aloss2:5.2411 exploreP:0.1604
Episode:246 meanR:153.8400 R:485.0000 rate:0.9700 aloss:1.3611 dloss:0.8379 aloss2:5.2106 exploreP:0.1533
Episode:247 meanR:158.3800 R:483.0000 rate:0.9

Episode:316 meanR:410.6800 R:152.0000 rate:0.3040 aloss:1.3141 dloss:1.3632 aloss2:3.5778 exploreP:0.0160
Episode:317 meanR:413.6100 R:500.0000 rate:1.0000 aloss:1.3165 dloss:1.3712 aloss2:3.5919 exploreP:0.0157
Episode:318 meanR:417.9500 R:500.0000 rate:1.0000 aloss:1.3166 dloss:1.3564 aloss2:3.6106 exploreP:0.0154
Episode:319 meanR:421.5900 R:500.0000 rate:1.0000 aloss:1.3259 dloss:1.2770 aloss2:3.6910 exploreP:0.0151
Episode:320 meanR:423.4300 R:414.0000 rate:0.8280 aloss:1.3102 dloss:1.4219 aloss2:3.6002 exploreP:0.0149
Episode:321 meanR:427.1700 R:500.0000 rate:1.0000 aloss:1.3206 dloss:1.3353 aloss2:3.6066 exploreP:0.0147
Episode:322 meanR:425.4500 R:211.0000 rate:0.4220 aloss:1.3144 dloss:1.3741 aloss2:3.6449 exploreP:0.0146
Episode:323 meanR:428.0500 R:500.0000 rate:1.0000 aloss:1.3179 dloss:1.3331 aloss2:3.6425 exploreP:0.0144
Episode:324 meanR:429.6000 R:278.0000 rate:0.5560 aloss:1.3121 dloss:1.3366 aloss2:3.6471 exploreP:0.0142
Episode:325 meanR:433.9600 R:500.0000 rate:1.0

...

Episode:3343 meanR:71.5900 R:37.0000 rate:0.0740 aloss:0.9712 dloss:0.6431 aloss2:64.8445 exploreP:0.0100
Episode:3344 meanR:70.7800 R:45.0000 rate:0.0900 aloss:0.9648 dloss:0.6617 aloss2:72.5479 exploreP:0.0100
Episode:3345 meanR:70.5800 R:54.0000 rate:0.1080 aloss:0.9734 dloss:0.6568 aloss2:72.2162 exploreP:0.0100
Episode:3346 meanR:70.4500 R:34.0000 rate:0.0680 aloss:0.9758 dloss:0.6535 aloss2:70.3102 exploreP:0.0100
Episode:3347 meanR:69.2600 R:48.0000 rate:0.0960 aloss:0.9754 dloss:0.6511 aloss2:63.9055 exploreP:0.0100
Episode:3348 meanR:68.3700 R:40.0000 rate:0.0800 aloss:0.9764 dloss:0.6438 aloss2:63.6508 exploreP:0.0100
Episode:3349 meanR:67.5600 R:50.0000 rate:0.1000 aloss:0.9695 dloss:0.6564 aloss2:65.3819 exploreP:0.0100
Episode:3350 meanR:67.1600 R:34.0000 rate:0.0680 aloss:0.9780 dloss:0.6483 aloss2:72.4175 exploreP:0.0100
Episode:3351 meanR:67.4000 R:60.0000 rate:0.1200 aloss:0.9755 dloss:0.6523 aloss2:73.3027 exploreP:0.0100
Episode:3352 meanR:66.1400 R:33.0000 rate:0.06

Episode:3421 meanR:60.0300 R:44.0000 rate:0.0880 aloss:0.9744 dloss:0.6392 aloss2:64.9021 exploreP:0.0100
Episode:3422 meanR:60.0300 R:49.0000 rate:0.0980 aloss:0.9780 dloss:0.6476 aloss2:68.0718 exploreP:0.0100
Episode:3423 meanR:60.2500 R:58.0000 rate:0.1160 aloss:0.9776 dloss:0.6506 aloss2:66.8568 exploreP:0.0100
Episode:3424 meanR:59.6400 R:71.0000 rate:0.1420 aloss:0.9779 dloss:0.6503 aloss2:69.1553 exploreP:0.0100
Episode:3425 meanR:58.6500 R:33.0000 rate:0.0660 aloss:0.9992 dloss:0.6720 aloss2:87.6127 exploreP:0.0100
Episode:3426 meanR:58.6200 R:37.0000 rate:0.0740 aloss:0.9835 dloss:0.6415 aloss2:71.9422 exploreP:0.0100
Episode:3427 meanR:58.8600 R:60.0000 rate:0.1200 aloss:0.9869 dloss:0.6503 aloss2:67.6944 exploreP:0.0100
Episode:3428 meanR:58.5300 R:43.0000 rate:0.0860 aloss:0.9686 dloss:0.6495 aloss2:81.6590 exploreP:0.0100
Episode:3429 meanR:58.6600 R:47.0000 rate:0.0940 aloss:0.9774 dloss:0.6601 aloss2:76.6161 exploreP:0.0100
Episode:3430 meanR:58.3800 R:31.0000 rate:0.06

Episode:3499 meanR:60.9300 R:49.0000 rate:0.0980 aloss:0.9823 dloss:0.6501 aloss2:66.1877 exploreP:0.0100
Episode:3500 meanR:60.6800 R:40.0000 rate:0.0800 aloss:0.9708 dloss:0.6452 aloss2:71.2691 exploreP:0.0100
Episode:3501 meanR:60.4700 R:36.0000 rate:0.0720 aloss:0.9853 dloss:0.6533 aloss2:79.3565 exploreP:0.0100
Episode:3502 meanR:60.5800 R:136.0000 rate:0.2720 aloss:0.9896 dloss:0.6484 aloss2:75.1563 exploreP:0.0100
Episode:3503 meanR:60.9200 R:79.0000 rate:0.1580 aloss:0.9836 dloss:0.6456 aloss2:73.7939 exploreP:0.0100
Episode:3504 meanR:61.3400 R:77.0000 rate:0.1540 aloss:0.9828 dloss:0.6480 aloss2:70.7853 exploreP:0.0100
Episode:3505 meanR:61.9500 R:117.0000 rate:0.2340 aloss:0.9815 dloss:0.6442 aloss2:72.7491 exploreP:0.0100
Episode:3506 meanR:62.0300 R:51.0000 rate:0.1020 aloss:0.9799 dloss:0.6482 aloss2:81.4774 exploreP:0.0100
Episode:3507 meanR:62.1200 R:54.0000 rate:0.1080 aloss:0.9805 dloss:0.6387 aloss2:67.5726 exploreP:0.0100
Episode:3508 meanR:62.4900 R:70.0000 rate:0.

Episode:3577 meanR:60.3600 R:72.0000 rate:0.1440 aloss:0.9807 dloss:0.6448 aloss2:74.5258 exploreP:0.0100
Episode:3578 meanR:60.3700 R:37.0000 rate:0.0740 aloss:0.9811 dloss:0.6456 aloss2:64.3031 exploreP:0.0100
Episode:3579 meanR:59.7200 R:56.0000 rate:0.1120 aloss:0.9773 dloss:0.6524 aloss2:74.8995 exploreP:0.0100
Episode:3580 meanR:59.8300 R:46.0000 rate:0.0920 aloss:0.9805 dloss:0.6519 aloss2:76.2543 exploreP:0.0100
Episode:3581 meanR:60.7400 R:129.0000 rate:0.2580 aloss:0.9843 dloss:0.6417 aloss2:71.1073 exploreP:0.0100
Episode:3582 meanR:60.0900 R:62.0000 rate:0.1240 aloss:0.9828 dloss:0.6336 aloss2:69.0971 exploreP:0.0100
Episode:3583 meanR:59.2200 R:46.0000 rate:0.0920 aloss:0.9953 dloss:0.6351 aloss2:78.0763 exploreP:0.0100
Episode:3584 meanR:59.3700 R:67.0000 rate:0.1340 aloss:0.9947 dloss:0.6407 aloss2:73.0472 exploreP:0.0100
Episode:3585 meanR:59.0100 R:31.0000 rate:0.0620 aloss:0.9810 dloss:0.6542 aloss2:78.8440 exploreP:0.0100
Episode:3586 meanR:59.0800 R:48.0000 rate:0.0

Episode:3655 meanR:64.0400 R:66.0000 rate:0.1320 aloss:0.9887 dloss:0.6372 aloss2:74.4229 exploreP:0.0100
Episode:3656 meanR:63.2500 R:48.0000 rate:0.0960 aloss:0.9863 dloss:0.6385 aloss2:79.0590 exploreP:0.0100
Episode:3657 meanR:63.2100 R:49.0000 rate:0.0980 aloss:0.9902 dloss:0.6407 aloss2:69.2619 exploreP:0.0100
Episode:3658 meanR:63.1400 R:39.0000 rate:0.0780 aloss:0.9875 dloss:0.6518 aloss2:69.3945 exploreP:0.0100
Episode:3659 meanR:61.9000 R:45.0000 rate:0.0900 aloss:0.9892 dloss:0.6333 aloss2:78.1412 exploreP:0.0100
Episode:3660 meanR:61.4200 R:55.0000 rate:0.1100 aloss:0.9894 dloss:0.6370 aloss2:75.4459 exploreP:0.0100
Episode:3661 meanR:60.8000 R:41.0000 rate:0.0820 aloss:0.9949 dloss:0.6430 aloss2:75.5347 exploreP:0.0100
Episode:3662 meanR:60.5800 R:40.0000 rate:0.0800 aloss:0.9927 dloss:0.6281 aloss2:84.2301 exploreP:0.0100
Episode:3663 meanR:60.5500 R:35.0000 rate:0.0700 aloss:0.9965 dloss:0.6286 aloss2:73.0420 exploreP:0.0100
Episode:3664 meanR:60.7000 R:51.0000 rate:0.10

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(gloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('G losses')`

In [ ]:
eps, arr = np.array(dloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 481.0
total_reward: 481.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.